In [ ]:
# Installing OpenStack Python client

In [ ]:
pip install python-openstackclient python-keystoneclient

In [ ]:
import sys
import yaml

from keystoneauth1.session import Session
from keystoneauth1.identity.v3.oidc import OidcAccessToken
from keystoneauth1.identity.v3.application_credential import ApplicationCredential


def load_access_token(token_file_path="/var/run/secrets/egi.eu/access_token"):
    """Reads access token from specified file"""
    try:
        return open(token_file_path, "r").read()
    except IOError:
        print("Reading from access token file failed.", file=sys.stderr)

def load_app_creds(creds_file_path="clouds.yaml"):
    """Reads application credentials from cloud.yaml file generated by OpenStack"""
    try:
         return yaml.safe_load(open(creds_file_path, "r"))
    except yaml.YAMLError:
        print("Reading from application credentials failed.", file=sys.stderr)


# Establishing session with OIDC token credential using OpenId
# 'auth_url' is server for authentization communication
# 'protocol' identifies a method used for authentization
# 'identity_provider' 
cloud_creds = OidcAccessToken(auth_url="https://identity.cloud.muni.cz/v3",
                              identity_provider="egi.eu",
                              protocol="openid",
                              access_token=load_access_token()
                             )


# This version is for 'clouds.yaml' credential files.
# It's also possible to use 'openrc' file. To use that
# one you have to set it to set required OS environment
# variables and then access them via os.getenv["OS_*"]
# e.g. OS_APPLICATION_CREDENTIAL_ID etc.

#parsed_app_creds = load_app_creds()
#openstack_app_creds = parsed_app_creds["clouds"]["openstack"]["auth"]


#cloud_creds = ApplicationCredential(
#                              auth_url=openstack_app_creds["auth_url"],
#                              application_credential_id=openstack_app_creds["application_credential_id"],
#                              application_credential_secret=openstack_app_creds["application_credential_secret"]
#                             )

cloud_session = Session(auth=cloud_creds)


In [ ]:
from keystoneclient.v3.client import Client
from openstack.connection import Connection

# OpenStack Keystone client for accessing 
# OpenStack parts not directly available 
# from primary API. We use keystone_client
# here to acquire details about our projects
# we can access. Once we have project details
# we can get scoped connection via primary API.
keystone_client = Client(session=cloud_session)

# OpenStack connection to primary API.
openstack_connection = Connection(session=cloud_session)

In [ ]:
# Print and save all projects where current
# token can be rescoped to
my_projects = keystone_client.auth.projects()

print(my_projects)

In [ ]:
# You have to bear in mind your application credentials/OIDC token
# has limited scope. That means you will have to use 'identity' 
# OpenStack proxy be specific when e.g. you are trying to list your
# projects i.e. you need to specify user ID.

# For example to see details about domains your projects are in
# you have to get ID of the domain and then get it through
# 'identity' OpenStack Python Proxy
for project in my_projects:
    print(openstack_connection.identity.get_domain(project.domain_id))


In [ ]:
# Now you are able scope to one of your projects you are entitled to access
scoped_cloud_creds = OidcAccessToken(auth_url="https://identity.cloud.muni.cz/v3",
                              identity_provider="egi.eu",
                              protocol="openid",
                              project_id=my_projects[1].id,
                              project_domain_id=my_projects[1].domain_id,
                              access_token=load_access_token()
                             )

scoped_cloud_session = Session(auth=scoped_cloud_creds)
scoped_openstack_connection = Connection(session=scoped_cloud_session)

# There are possibilities to use methods like 
# https://docs.openstack.org/openstacksdk/latest/user/connection.html#openstack.connection.Connection.list_projects
#
# However, these require admin credentials. That means you won't be able to use them

In [ ]:
# You can check the current project ID you are currently operanting in via
# 'current_project' or 'current_project_id' attribute

# If you wish to change your project you have to specify it
# during credentials setting by 'project_id'/'project_name'
# and 'project_domain_name'/'project_domain_id'

print(scoped_openstack_connection.current_project_id)

In [ ]:
# First we create a SSH key to get access to our created VMs.
# If you wish to use the existing key, skip this step and write
# key name into`key_name` parameter in `create_server` method.

#You can use your own generated public key and specify it in `public_key`
#parameter to create key-pair or let OpenStack generate one for you.

#Generating key-pair using existing public key
#scoped_openstack_connection.create_keypair("mykey2", public_key="ssh-rsa ....")

#If you let OpenStack generate your key-pair you will need to save your
#private and public keys for later use
new_keypair = scoped_openstack_connection.create_keypair("mykey")

#Both keys are available as string attributes in created
#key-pair object
#print(new_keypair.private_key)
#print(new_keypair.public_key)

In [ ]:
# Before creating the VM itself it is good to consider
# what kind of network traffic we want to allow for our VM.

# OpenStack allows to configure this by Security Groups.
# There is usually a default security group present in OpenStack.
# If the default one is not sufficient we can specify a new one.

new_security_group = scoped_openstack_connection.create_security_group("New VM security group", "Example security group")

# Once new security group is created we can specify what traffic we want to allow for 
# our new VM.
# In this setup we want to allow:
#                                  1. ICMP ingress - for ping test
#                                  2. SSH ingress - for accessing VM
#
# A new security group creates IPv4 egress rule by default

# Rule for accepting incoming SSH traffic
scoped_openstack_connection.create_security_group_rule(new_security_group.id, 22, 22, protocol="tcp", direction="ingress", description="SSH rule")
# Rule for ICMP traffic
scoped_openstack_connection.create_security_group_rule(new_security_group.id, protocol="icmp", description="ICMP rule")

In [ ]:
import time

# Now we can gather all required information and perform
# tasks through the connection and its service proxies.
# More in https://docs.openstack.org/openstacksdk/latest/user/index.html#service-proxies

# If we want to perform certain tasks during VM creation e.g. installing dependencies
# we can use cloud-init mechanism which is basically script which is passed to OpenStack
# during VM creation and is executed. You can use either file_handle of script or pass it 
# as string.

# Beware!! cloud-init is sensitive about script beginning. There can be no
# space or new line before #!... part. Otherwise it will fail.

cloud_init_script = \
"""#!/bin/bash

apt-get update
apt-get upgrade -y

curl https://bootstrap.pypa.io/get-pip.py > get-pip.py

python3 get-pip.py

python3 -m pip install ansible
"""

selected_image = scoped_openstack_connection.compute.find_image("debian-11-x86_64")

selected_flavor = scoped_openstack_connection.compute.find_flavor("standard.small")

selected_network = scoped_openstack_connection.network.find_network("147-251-115-pers-proj-net")



# Creating new volume to attach to VM. Size of the volume is 80GB.
new_volume = scoped_openstack_connection.create_volume(80)


new_vm = scoped_openstack_connection.create_server("New VM",
                                            image=selected_image,
                                            flavor=selected_flavor,
                                            network=selected_network,
                                            userdata=cloud_init_script,
                                            key_name="mykey",
                                            volumes=[new_volume],
                                            security_groups=[new_security_group.name]
                                            )

# Waiting until all is ready
time.sleep(10)

# Refreshing VM object to get the current state
new_vm = scoped_openstack_connection.compute.find_server(new_vm.id)


In [ ]:
# Now we can create new floating IP address.
# create_floating_ip also accepts additional
# paramaters for associating new FIP with
# existing server/port.
new_fip = scoped_openstack_connection.create_floating_ip(network="public-muni-147-251-115-PERSONAL")

In [ ]:
# Next step is to add new FIP to new VM
# to get access through SSH.

# add_ip_list requires actual address string
# or list of strings of address
scoped_openstack_connection.add_ip_list(new_vm, new_fip.floating_ip_address)

In [ ]:
# This example shows how to resize existing VM
# to different selected flavor

new_flavor = scoped_openstack_connection.compute.find_flavor("standard.medium")

scoped_openstack_connection.compute.resize_server(new_vm, new_flavor)

# Now we wait a bit for OpenStack to prepare required changes
time.sleep(15)


#This confirmes resize change

scoped_openstack_connection.compute.confirm_server_resize(new_vm)

#This reverts resize change

#scoped_openstack_connection.compute.revert_server_resize(new_vm)


In [ ]:
# We can suspend VM
scoped_openstack_connection.compute.suspend_server(new_vm)


# Now we wait a bit for OpenStack to prepare required changes
time.sleep(15)

# After that we can turn it on again

scoped_openstack_connection.compute.resume_server(new_vm)

time.sleep(15)

# Or we can combine both and reboot VM
# using HARD/SOFT reboot

scoped_openstack_connection.compute.reboot_server(new_vm, reboot_type="SOFT")


In [ ]:
# If you need to detach FIP from current server
# you can use following method

scoped_openstack_connection.detach_ip_from_server(new_vm, new_fip.id)

In [ ]:
# You backup your attached volume
attached_volume = new_vm.attached_volumes[0]

# OpenStack recommends to suspend VM and then backup volume.
# You can use `force` parameter to bypass it.

scoped_openstack_connection.create_volume_backup(attached_volume.id, name="My Backup", force=True)


In [ ]:
# After finishing the work we can release allocated
# resources

scoped_openstack_connection.delete_server(new_vm.id)
scoped_openstack_connection.delete_floating_ip(new_fip)

In [ ]:
# If you do not wish to use generated key-pair anymore
# you can simply delete it
scoped_openstack_connection.delete_keypair("mykey")